In [42]:
#!/usr/bin/env python3
"""
FoodieBot Phase 1: Product Data Generation & Database Setup
AI Food Agent Assignment - Tecnvirons Pvt LTD

This module handles:
1. Fast food product data generation using AI
2. SQLite database setup and schema creation
3. Data validation and insertion
4. Database indexing for optimal performance
"""

import sqlite3
import json
import random
from datetime import datetime
from typing import List, Dict, Any, Optional
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class FastFoodProductGenerator:
    """
    Generates realistic fast food products with proper categorization and attributes
    """

    def __init__(self):
        self.categories = {
            "Burgers": 10,
            "Pizza": 10,
            "Fried Chicken": 10,
            "Tacos & Wraps": 10,
            "Sides & Appetizers": 10,
            "Beverages": 10,
            "Desserts": 10,
            "Salads & Healthy Options": 10,
            "Breakfast Items": 10,
            "Limited Time Specials": 10
        }

        # Pre-defined ingredients by category for realistic combinations
        self.ingredients_by_category = {
            "Burgers": [
                ["beef patty", "cheddar cheese", "lettuce", "tomato", "sesame bun", "mayo"],
                ["chicken breast", "swiss cheese", "avocado", "brioche bun", "chipotle mayo"],
                ["black bean patty", "pepper jack cheese", "sprouts", "whole wheat bun", "avocado spread"],
                ["turkey patty", "bacon", "lettuce", "tomato", "sourdough bun", "ranch"],
                ["beef patty", "gochujang sauce", "kimchi", "brioche bun", "sriracha aioli"],
                ["portobello mushroom", "goat cheese", "arugula", "pretzel bun", "herb aioli"],
                ["lamb patty", "feta cheese", "cucumber", "pita bun", "tzatziki"],
                ["salmon burger", "cream cheese", "capers", "everything bagel", "dill sauce"],
                ["impossible patty", "vegan cheese", "lettuce", "tomato", "vegan bun", "vegan mayo"],
                ["bison patty", "blue cheese", "onion rings", "brioche bun", "buffalo sauce"]
            ],
            "Pizza": [
                ["tomato sauce", "mozzarella", "pepperoni", "pizza dough"],
                ["white sauce", "mozzarella", "grilled chicken", "spinach", "pizza dough"],
                ["tomato sauce", "mozzarella", "mushrooms", "bell peppers", "olives", "pizza dough"],
                ["bbq sauce", "mozzarella", "pulled pork", "red onions", "pizza dough"],
                ["pesto sauce", "mozzarella", "sun-dried tomatoes", "goat cheese", "pizza dough"],
                ["tomato sauce", "mozzarella", "italian sausage", "peppers", "pizza dough"],
                ["buffalo sauce", "mozzarella", "chicken", "celery", "blue cheese", "pizza dough"],
                ["tomato sauce", "mozzarella", "prosciutto", "arugula", "parmesan", "pizza dough"],
                ["white sauce", "mozzarella", "shrimp", "garlic", "herbs", "pizza dough"],
                ["tomato sauce", "vegan cheese", "vegetables", "vegan dough"]
            ],
            "Fried Chicken": [
                ["chicken wings", "buffalo sauce", "celery", "blue cheese dip"],
                ["chicken tenders", "honey mustard", "french fries"],
                ["chicken breast", "pickles", "mayo", "brioche bun"],
                ["chicken thighs", "korean glaze", "coleslaw"],
                ["chicken wings", "bbq sauce", "ranch dip"],
                ["chicken breast", "nashville hot seasoning", "pickle chips"],
                ["chicken tenders", "maple syrup", "waffle"],
                ["chicken wings", "garlic parmesan", "marinara sauce"],
                ["chicken breast", "sriracha mayo", "asian slaw", "steamed bun"],
                ["chicken strips", "buttermilk ranch", "texas toast"]
            ],
            "Tacos & Wraps": [
                ["corn tortilla", "beef", "onions", "cilantro", "salsa verde"],
                ["flour tortilla", "chicken", "lettuce", "tomato", "cheese", "sour cream"],
                ["corn tortilla", "fish", "cabbage slaw", "chipotle mayo", "lime"],
                ["flour tortilla", "carnitas", "pickled onions", "salsa roja"],
                ["spinach wrap", "turkey", "avocado", "sprouts", "hummus"],
                ["flour tortilla", "shrimp", "mango salsa", "cilantro lime crema"],
                ["corn tortilla", "chorizo", "eggs", "cheese", "hot sauce"],
                ["flour tortilla", "black beans", "rice", "guacamole", "pico de gallo"],
                ["corn tortilla", "pastor", "pineapple", "onions", "cilantro"],
                ["flour tortilla", "grilled vegetables", "quinoa", "tahini sauce"]
            ],
            "Sides & Appetizers": [
                ["potatoes", "salt", "oil"],
                ["onions", "flour", "buttermilk", "spices"],
                ["mozzarella", "marinara sauce", "breadcrumbs"],
                ["chicken wings", "buffalo sauce"],
                ["tortilla chips", "cheese sauce", "jalapeños"],
                ["potatoes", "cheese", "bacon", "sour cream"],
                ["jalapeños", "cream cheese", "bacon"],
                ["sweet potatoes", "cinnamon", "oil"],
                ["mac cheese", "breadcrumbs", "truffle oil"],
                ["cauliflower", "buffalo sauce", "ranch"]
            ],
            "Beverages": [
                ["cola syrup", "carbonated water", "ice"],
                ["vanilla ice cream", "milk", "whipped cream"],
                ["coffee", "milk", "sugar", "ice"],
                ["fruit juice", "sparkling water", "fresh fruit"],
                ["tea", "milk", "tapioca pearls", "sugar"],
                ["chocolate syrup", "milk", "ice cream", "whipped cream"],
                ["fresh fruit", "yogurt", "honey"],
                ["espresso", "steamed milk", "foam"],
                ["lemonade", "fresh lemons", "sugar", "mint"],
                ["coconut water", "pineapple juice", "ice"]
            ],
            "Desserts": [
                ["chocolate chips", "cookie dough", "vanilla ice cream"],
                ["apples", "cinnamon", "pastry", "caramel sauce"],
                ["chocolate cake", "fudge", "ice cream"],
                ["vanilla ice cream", "hot fudge", "whipped cream", "cherry"],
                ["cream cheese", "graham crackers", "berries"],
                ["chocolate", "marshmallow", "graham crackers"],
                ["churros", "cinnamon sugar", "chocolate sauce"],
                ["donuts", "glaze", "sprinkles"],
                ["brownie", "vanilla ice cream", "caramel"],
                ["lemon curd", "pastry", "meringue"]
            ],
            "Salads & Healthy Options": [
                ["mixed greens", "chicken breast", "caesar dressing", "parmesan", "croutons"],
                ["spinach", "strawberries", "feta cheese", "balsamic vinaigrette"],
                ["kale", "quinoa", "avocado", "lemon dressing"],
                ["arugula", "grilled salmon", "cherry tomatoes", "olive oil"],
                ["mixed greens", "turkey", "cranberries", "walnuts", "honey mustard"],
                ["romaine", "black beans", "corn", "avocado", "lime dressing"],
                ["spinach", "grilled shrimp", "mango", "citrus vinaigrette"],
                ["kale", "roasted vegetables", "tahini dressing"],
                ["mixed greens", "tuna", "eggs", "olives", "vinaigrette"],
                ["arugula", "beets", "goat cheese", "candied pecans", "balsamic"]
            ],
            "Breakfast Items": [
                ["eggs", "bacon", "hash browns", "toast"],
                ["pancakes", "maple syrup", "butter", "berries"],
                ["english muffin", "canadian bacon", "eggs", "hollandaise"],
                ["oatmeal", "fresh fruit", "honey", "nuts"],
                ["eggs", "cheese", "sausage", "tortilla"],
                ["french toast", "cinnamon", "maple syrup", "powdered sugar"],
                ["yogurt", "granola", "fresh berries", "honey"],
                ["eggs", "avocado", "toast", "everything seasoning"],
                ["breakfast sandwich", "egg", "cheese", "english muffin"],
                ["acai", "granola", "banana", "coconut"]
            ],
            "Limited Time Specials": [
                ["seasonal ingredients", "special sauce", "limited edition bun"],
                ["holiday spices", "festive garnish", "premium ingredients"],
                ["exotic proteins", "fusion sauces", "artisan bread"],
                ["local ingredients", "chef's special sauce", "handcrafted elements"],
                ["imported cheese", "truffle oil", "gourmet toppings"],
                ["organic vegetables", "house-made dressing", "farm-fresh ingredients"],
                ["aged meats", "signature marinade", "special preparation"],
                ["rare spices", "international flavors", "unique presentation"],
                ["premium seafood", "chef's sauce", "artisanal sides"],
                ["seasonal fruits", "specialty cream", "limited-time pastry"]
            ]
        }

        self.mood_tags_options = [
            ["comfort", "indulgent"], ["adventurous", "exotic"], ["healthy", "fresh"],
            ["spicy", "bold"], ["sweet", "satisfying"], ["savory", "rich"],
            ["light", "refreshing"], ["hearty", "filling"], ["nostalgic", "classic"],
            ["innovative", "modern"], ["cozy", "warm"], ["energizing", "vibrant"]
        ]

        self.dietary_tags_options = [
            ["vegetarian"], ["vegan"], ["gluten-free"], ["dairy-free"],
            ["spicy"], ["mild"], ["fusion"], ["classic"], ["organic"],
            ["contains_gluten"], ["contains_dairy"], ["contains_nuts"],
            ["low-carb"], ["high-protein"], ["keto-friendly"]
        ]

    def generate_product_name(self, category: str, ingredients: List[str]) -> str:
        """Generate creative product names based on category and ingredients"""

        name_patterns = {
            "Burgers": [
                "{adj} {protein} {style} Burger",
                "The {adj} {ingredient} Stack",
                "{fusion} {protein} Deluxe",
                "Ultimate {ingredient} Burger"
            ],
            "Pizza": [
                "{adj} {ingredient} Pizza",
                "The {style} Special",
                "{fusion} {ingredient} Pie",
                "Classic {ingredient} Slice"
            ],
            "Fried Chicken": [
                "{adj} {style} {cut}",
                "{fusion} {cut} Combo",
                "Crispy {ingredient} {cut}",
                "Nashville {style} {cut}"
            ],
            "Tacos & Wraps": [
                "{fusion} {protein} Tacos",
                "{adj} {ingredient} Wrap",
                "Street Style {protein} Tacos",
                "Grilled {ingredient} Burrito"
            ]
        }

        adjectives = ["Spicy", "Golden", "Crispy", "Smoky", "Zesty", "Bold", "Premium", "Fire", "Ultimate", "Supreme"]
        fusion_words = ["Korean", "Mexican", "Italian", "Asian", "Mediterranean", "Cajun", "BBQ", "Buffalo", "Teriyaki", "Chipotle"]

        # Simplified naming for other categories
        if category not in name_patterns:
            return f"{random.choice(adjectives)} {random.choice(ingredients).title()} {category.split()[0][:-1]}"

        pattern = random.choice(name_patterns[category])

        return pattern.format(
            adj=random.choice(adjectives),
            protein=random.choice(["Beef", "Chicken", "Turkey", "Fish", "Veggie"]),
            ingredient=random.choice(ingredients).title(),
            style=random.choice(["Fusion", "Classic", "Gourmet", "Crispy", "Grilled"]),
            fusion=random.choice(fusion_words),
            cut=random.choice(["Wings", "Tenders", "Sandwich", "Bowl"])
        )

    def generate_products(self) -> List[Dict[str, Any]]:
        """Generate 100 fast food products across all categories"""
        products = []
        product_id_counter = 1

        logger.info("Starting product generation...")

        for category, count in self.categories.items():
            logger.info(f"Generating {count} products for {category}")

            for i in range(count):
                # Get ingredients for this category
                ingredients = random.choice(self.ingredients_by_category[category])

                # Generate product data
                product = {
                    "product_id": f"FF{product_id_counter:03d}",
                    "name": self.generate_product_name(category, ingredients),
                    "category": category,
                    "description": self.generate_description(category, ingredients),
                    "ingredients": ingredients,
                    "price": round(random.uniform(4.99, 18.99), 2),
                    "calories": random.randint(200, 1200),
                    "prep_time": f"{random.randint(5, 15)}-{random.randint(15, 25)} mins",
                    "dietary_tags": random.choice(self.dietary_tags_options),
                    "mood_tags": random.choice(self.mood_tags_options),
                    "allergens": self.generate_allergens(ingredients),
                    "popularity_score": random.randint(60, 95),
                    "chef_special": random.choice([True, False]) and random.random() < 0.3,
                    "limited_time": category == "Limited Time Specials" or random.random() < 0.1,
                    "spice_level": random.randint(0, 10) if any(word in str(ingredients) for word in ["spicy", "hot", "chili", "jalapeño", "sriracha"]) else random.randint(0, 3),
                    "image_prompt": self.generate_image_prompt(category, ingredients),
                    "created_at": datetime.now().isoformat(),
                    "last_updated": datetime.now().isoformat()
                }

                products.append(product)
                product_id_counter += 1

        logger.info(f"Generated {len(products)} products successfully")
        return products

    def generate_description(self, category: str, ingredients: List[str]) -> str:
        """Generate realistic product descriptions"""
        descriptions = {
            "Burgers": f"Juicy {ingredients[0]} perfectly seasoned and grilled, topped with {', '.join(ingredients[1:3])}, served on a fresh {ingredients[-2]} with our signature {ingredients[-1]}",
            "Pizza": f"Hand-tossed dough topped with {ingredients[0]}, melted {ingredients[1]}, and premium {', '.join(ingredients[2:-1])}",
            "Fried Chicken": f"Crispy golden {ingredients[0]} served with {ingredients[1]} and accompanied by {', '.join(ingredients[2:])}",
            "Tacos & Wraps": f"Fresh {ingredients[0]} filled with seasoned {ingredients[1]}, topped with {', '.join(ingredients[2:4])}, and finished with our house-made {ingredients[-1]}",
        }

        if category in descriptions:
            return descriptions[category]
        else:
            return f"Delicious {category.lower()} featuring {', '.join(ingredients[:3])} and more premium ingredients"

    def generate_allergens(self, ingredients: List[str]) -> List[str]:
        """Generate allergen information based on ingredients"""
        allergen_map = {
            "gluten": ["bun", "bread", "dough", "flour", "wheat", "toast"],
            "dairy": ["cheese", "milk", "cream", "butter", "mayo", "ranch"],
            "eggs": ["mayo", "egg", "aioli"],
            "soy": ["soy", "tofu", "miso", "teriyaki"],
            "nuts": ["nuts", "peanut", "almond", "walnut"],
            "shellfish": ["shrimp", "crab", "lobster"],
            "fish": ["salmon", "tuna", "fish", "anchovies"]
        }

        allergens = []
        ingredient_str = " ".join(ingredients).lower()

        for allergen, triggers in allergen_map.items():
            if any(trigger in ingredient_str for trigger in triggers):
                allergens.append(allergen)

        return allergens

    def generate_image_prompt(self, category: str, ingredients: List[str]) -> str:
        """Generate image prompts for product visualization"""
        return f"{category.lower()} with {', '.join(ingredients[:3])}, professional food photography, appetizing presentation"

class FoodieBotDatabase:
    """
    Database manager for FoodieBot with optimized schema and indexing
    """

    def __init__(self, db_path: str = "foodiebot.db"):
        self.db_path = db_path
        self.init_database()

    def init_database(self):
        """Initialize database with optimized schema and indexes"""
        logger.info("Initializing database...")

        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()

            # Create products table with optimized schema
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS products (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    product_id TEXT UNIQUE NOT NULL,
                    name TEXT NOT NULL,
                    category TEXT NOT NULL,
                    description TEXT,
                    ingredients TEXT, -- JSON array as string
                    price REAL NOT NULL,
                    calories INTEGER,
                    prep_time TEXT,
                    dietary_tags TEXT, -- JSON array as string
                    mood_tags TEXT, -- JSON array as string
                    allergens TEXT, -- JSON array as string
                    popularity_score INTEGER DEFAULT 0,
                    chef_special BOOLEAN DEFAULT FALSE,
                    limited_time BOOLEAN DEFAULT FALSE,
                    spice_level INTEGER DEFAULT 0,
                    image_prompt TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    active BOOLEAN DEFAULT TRUE
                )
            ''')

            # Create conversation logs table
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS conversation_logs (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    session_id TEXT NOT NULL,
                    user_message TEXT,
                    bot_response TEXT,
                    interest_score INTEGER DEFAULT 0,
                    recommended_products TEXT, -- JSON array of product IDs
                    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    user_preferences TEXT -- JSON object
                )
            ''')

            # Create analytics table
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS analytics (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    product_id TEXT,
                    event_type TEXT, -- 'view', 'recommend', 'click', 'order'
                    session_id TEXT,
                    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    metadata TEXT, -- JSON for additional data
                    FOREIGN KEY (product_id) REFERENCES products (product_id)
                )
            ''')

            # Create performance indexes
            self._create_indexes(cursor)

            conn.commit()

        logger.info("Database initialized successfully")

    def _create_indexes(self, cursor):
        """Create indexes for optimal query performance"""
        indexes = [
            "CREATE INDEX IF NOT EXISTS idx_products_category ON products(category)",
            "CREATE INDEX IF NOT EXISTS idx_products_price ON products(price)",
            "CREATE INDEX IF NOT EXISTS idx_products_popularity ON products(popularity_score)",
            "CREATE INDEX IF NOT EXISTS idx_products_spice_level ON products(spice_level)",
            "CREATE INDEX IF NOT EXISTS idx_products_chef_special ON products(chef_special)",
            "CREATE INDEX IF NOT EXISTS idx_products_limited_time ON products(limited_time)",
            "CREATE INDEX IF NOT EXISTS idx_products_active ON products(active)",
            "CREATE INDEX IF NOT EXISTS idx_conversation_session ON conversation_logs(session_id)",
            "CREATE INDEX IF NOT EXISTS idx_conversation_timestamp ON conversation_logs(timestamp)",
            "CREATE INDEX IF NOT EXISTS idx_analytics_product ON analytics(product_id)",
            "CREATE INDEX IF NOT EXISTS idx_analytics_event ON analytics(event_type)",
            "CREATE INDEX IF NOT EXISTS idx_analytics_timestamp ON analytics(timestamp)"
        ]

        for index in indexes:
            cursor.execute(index)

    def insert_products(self, products: List[Dict[str, Any]]) -> bool:
        """Insert products into database with validation"""
        logger.info(f"Inserting {len(products)} products into database...")

        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()

                for product in products:
                    cursor.execute('''
                        INSERT OR REPLACE INTO products (
                            product_id, name, category, description, ingredients,
                            price, calories, prep_time, dietary_tags, mood_tags,
                            allergens, popularity_score, chef_special, limited_time,
                            spice_level, image_prompt, created_at, last_updated
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        product['product_id'],
                        product['name'],
                        product['category'],
                        product['description'],
                        json.dumps(product['ingredients']),
                        product['price'],
                        product['calories'],
                        product['prep_time'],
                        json.dumps(product['dietary_tags']),
                        json.dumps(product['mood_tags']),
                        json.dumps(product['allergens']),
                        product['popularity_score'],
                        product['chef_special'],
                        product['limited_time'],
                        product['spice_level'],
                        product['image_prompt'],
                        product['created_at'],
                        product['last_updated']
                    ))

                conn.commit()
                logger.info("Products inserted successfully")
                return True

        except Exception as e:
            logger.error(f"Error inserting products: {e}")
            return False

    def get_products_by_category(self, category: str) -> List[Dict[str, Any]]:
        """Get products by category with JSON parsing"""
        with sqlite3.connect(self.db_path) as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()

            cursor.execute('''
                SELECT * FROM products
                WHERE category = ? AND active = TRUE
                ORDER BY popularity_score DESC
            ''', (category,))

            rows = cursor.fetchall()
            products = []

            for row in rows:
                product = dict(row)
                # Parse JSON fields
                product['ingredients'] = json.loads(product['ingredients'])
                product['dietary_tags'] = json.loads(product['dietary_tags'])
                product['mood_tags'] = json.loads(product['mood_tags'])
                product['allergens'] = json.loads(product['allergens'])
                products.append(product)

            return products

    def search_products(self, **kwargs) -> List[Dict[str, Any]]:
        """Advanced product search with multiple filters"""
        with sqlite3.connect(self.db_path) as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()

            query = "SELECT * FROM products WHERE active = TRUE"
            params = []

            if 'category' in kwargs:
                query += " AND category = ?"
                params.append(kwargs['category'])

            if 'max_price' in kwargs:
                query += " AND price <= ?"
                params.append(kwargs['max_price'])

            if 'min_price' in kwargs:
                query += " AND price >= ?"
                params.append(kwargs['min_price'])

            if 'max_spice_level' in kwargs:
                query += " AND spice_level <= ?"
                params.append(kwargs['max_spice_level'])

            if 'chef_special' in kwargs:
                query += " AND chef_special = ?"
                params.append(kwargs['chef_special'])

            query += " ORDER BY popularity_score DESC LIMIT 50"

            cursor.execute(query, params)
            rows = cursor.fetchall()

            products = []
            for row in rows:
                product = dict(row)
                product['ingredients'] = json.loads(product['ingredients'])
                product['dietary_tags'] = json.loads(product['dietary_tags'])
                product['mood_tags'] = json.loads(product['mood_tags'])
                product['allergens'] = json.loads(product['allergens'])
                products.append(product)

            return products

    def get_database_stats(self) -> Dict[str, Any]:
        """Get database statistics for monitoring"""
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()

            stats = {}

            # Product count by category
            cursor.execute('''
                SELECT category, COUNT(*) as count
                FROM products
                WHERE active = TRUE
                GROUP BY category
            ''')
            stats['products_by_category'] = dict(cursor.fetchall())

            # Total products
            cursor.execute('SELECT COUNT(*) FROM products WHERE active = TRUE')
            stats['total_products'] = cursor.fetchone()[0]

            # Price statistics
            cursor.execute('''
                SELECT MIN(price) as min_price, MAX(price) as max_price,
                       AVG(price) as avg_price FROM products WHERE active = TRUE
            ''')
            price_stats = cursor.fetchone()
            stats['price_stats'] = {
                'min': price_stats[0],
                'max': price_stats[1],
                'average': round(price_stats[2], 2)
            }

            return stats

def main():
    """Main execution function for Phase 1"""
    logger.info("Starting FoodieBot Phase 1 - Product Generation & Database Setup")

    # Initialize components
    generator = FastFoodProductGenerator()
    # Remove old database to ensure schema update
    db_path = "foodiebot.db"
    if os.path.exists(db_path):
      os.remove(db_path)
    database = FoodieBotDatabase()

    # Generate products
    products = generator.generate_products()

    # Save products to JSON for backup
    with open('generated_products.json', 'w') as f:
        json.dump(products, f, indent=2)
    logger.info("Products saved to generated_products.json")

    # Insert into database
    success = database.insert_products(products)

    if success:
        # Display database statistics
        stats = database.get_database_stats()
        logger.info("Database Statistics:")
        logger.info(f"Total Products: {stats['total_products']}")
        logger.info("Products by Category:")
        for category, count in stats['products_by_category'].items():
            logger.info(f"  {category}: {count}")
        logger.info(f"Price Range: ${stats['price_stats']['min']:.2f} - ${stats['price_stats']['max']:.2f}")
        logger.info(f"Average Price: ${stats['price_stats']['average']:.2f}")

        # Test some database queries
        logger.info("\nTesting database queries...")

        # Test category search
        burgers = database.get_products_by_category("Burgers")
        logger.info(f"Found {len(burgers)} burgers")

        # Test advanced search
        spicy_under_10 = database.search_products(max_price=10.00, max_spice_level=10)
        logger.info(f"Found {len(spicy_under_10)} items under $10")

        logger.info("Phase 1 completed successfully!")
        return True
    else:
        logger.error("Phase 1 failed during database insertion")
        return False

if __name__ == "__main__":
    main()

In [46]:
import sqlite3
import json
import re
import time
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, field
import requests
import os
from groq import Groq
import logging
from pathlib import Path

# Only import streamlit when actually running in streamlit
try:
    import streamlit as st
    STREAMLIT_AVAILABLE = True
except ImportError:
    STREAMLIT_AVAILABLE = False

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('foodiebot.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

@dataclass
class ConversationContext:
    """Enhanced conversation context with better memory management"""
    user_preferences: Dict[str, Any] = field(default_factory=dict)
    dietary_restrictions: List[str] = field(default_factory=list)
    budget_range: Optional[Tuple[float, float]] = None
    mood_tags: List[str] = field(default_factory=list)
    mentioned_products: List[str] = field(default_factory=list)
    conversation_history: List[Dict[str, str]] = field(default_factory=list)
    current_interest_score: float = 0.0
    session_start_time: datetime = field(default_factory=datetime.now)
    last_response_time: Optional[datetime] = None
    current_category_focus: Optional[str] = None
    last_recommendations: List[str] = field(default_factory=list)
    user_engagement_patterns: Dict[str, int] = field(default_factory=dict)
    conversation_depth: int = 0
    last_interest_boost: Optional[datetime] = None
    preferred_price_range: Tuple[float, float] = (0.0, 50.0)

    def update_engagement_pattern(self, pattern: str):
        """Track user engagement patterns"""
        self.user_engagement_patterns[pattern] = self.user_engagement_patterns.get(pattern, 0) + 1

    def get_dominant_preferences(self, top_n: int = 3) -> List[str]:
        """Get user's top preferences"""
        if not self.user_preferences:
            return []

        all_prefs = []
        for key, value in self.user_preferences.items():
            if isinstance(value, list):
                all_prefs.extend(value)
            else:
                all_prefs.append(str(value))

        return list(set(all_prefs))[:top_n]

class InterestScoringEngine:
    """
    Advanced interest scoring with temporal awareness and pattern learning
    """

    def __init__(self):
        # Enhanced scoring factors
        self.ENGAGEMENT_FACTORS = {
            'specific_preferences': 15,
            'dietary_restrictions': 10,
            'budget_mention': 8,
            'mood_indication': 20,
            'question_asking': 12,
            'enthusiasm_words': 10,
            'price_inquiry': 25,
            'order_intent': 35,
            'basic_engagement': 5,
            'food_mention': 12,
            'category_mention': 18,
            'comparison_request': 20,
            'follow_up_question': 15,
            'detailed_description': 22,
            'personal_story': 25,
            'urgency_indicators': 28,
        }

        self.NEGATIVE_FACTORS = {
            'hesitation': -12,
            'budget_concern': -18,
            'dietary_conflict': -25,
            'rejection': -30,
            'delay_response': -8,
            'repetition_boredom': -15,
            'confusion_indicators': -10,
            'distraction_signals': -20,
        }

        # Enhanced pattern recognition
        self.patterns = {
            'specific_preferences': [
                r'i love', r'i really like', r'my favorite', r'i prefer',
                r'i enjoy', r'craving', r'want something', r'i like',
                r'love', r'favorite', r'prefer', r'enjoy', r'adore'
            ],
            'dietary_restrictions': [
                r'vegetarian', r'vegan', r'no meat', r'gluten free', r'gluten-free',
                r'lactose intolerant', r'allergic to', r'can\'t eat', r'avoid',
                r'keto', r'paleo', r'dairy free', r'dairy-free'
            ],
            'budget_mention': [
                r'under \$?\d+', r'less than \$?\d+', r'budget', r'cheap',
                r'affordable', r'inexpensive', r'money', r'cost', r'price',
                r'around \$?\d+', r'about \$?\d+'
            ],
            'mood_indication': [
                r'feeling', r'in the mood', r'want something', r'craving',
                r'looking for', r'need something', r'hungry for', r'mood',
                r'today i want', r'right now i'
            ],
            'question_asking': [
                r'what\'?s', r'how much', r'how spicy', r'what ingredients',
                r'can you tell me', r'do you have', r'is it', r'does it contain',
                r'\?', r'what', r'how', r'where', r'when', r'why', r'which'
            ],
            'enthusiasm_words': [
                r'amazing', r'perfect', r'awesome', r'great', r'excellent',
                r'love it', r'sounds good', r'delicious', r'fantastic',
                r'wonderful', r'good', r'nice', r'cool', r'yes', r'definitely',
                r'absolutely', r'incredible', r'outstanding'
            ],
            'price_inquiry': [
                r'how much', r'what\'?s the price', r'cost', r'expensive',
                r'price', r'dollar', r'\$', r'money', r'pay', r'worth it'
            ],
            'order_intent': [
                r'i\'?ll take', r'add to cart', r'order', r'i want that',
                r'get me', r'i\'?ll have', r'sign me up', r'let\'?s go with',
                r'buy', r'purchase', r'take it', r'sold', r'i\'?m getting'
            ],
            'food_mention': [
                r'burger', r'pizza', r'chicken', r'beef', r'fish', r'salad',
                r'sandwich', r'fries', r'drink', r'food', r'eat', r'hungry',
                r'spicy', r'sweet', r'salty', r'hot', r'cold', r'meal'
            ],
            'category_mention': [
                r'burger', r'pizza', r'chicken', r'beef', r'salad', r'sandwich',
                r'pasta', r'mexican', r'asian', r'italian', r'dessert', r'drink',
                r'breakfast', r'lunch', r'dinner', r'snack'
            ],
            'comparison_request': [
                r'what\'?s better', r'compare', r'difference between', r'vs',
                r'rather have', r'instead of', r'alternative', r'something else',
                r'which is', r'or should i'
            ],
            'follow_up_question': [
                r'also', r'and what about', r'what else', r'anything else',
                r'more options', r'other', r'besides that'
            ],
            'detailed_description': [
                r'tell me more', r'describe', r'what\'?s in it', r'ingredients',
                r'how is it made', r'preparation', r'details'
            ],
            'personal_story': [
                r'last time', r'usually', r'always', r'never', r'my friend',
                r'i remember', r'back home', r'growing up'
            ],
            'urgency_indicators': [
                r'right now', r'immediately', r'asap', r'quick', r'fast',
                r'in a hurry', r'soon', r'today'
            ],
            'hesitation': [
                r'maybe', r'not sure', r'i guess', r'perhaps', r'might',
                r'i don\'?t know', r'hmm', r'uncertain', r'unsure', r'think'
            ],
            'budget_concern': [
                r'too expensive', r'too much', r'can\'?t afford', r'pricey',
                r'costly', r'over budget', r'too dear', r'expensive'
            ],
            'rejection': [
                r'i don\'?t like', r'hate', r'not a fan', r'don\'?t want',
                r'no thanks', r'pass', r'not interested', r'skip', r'nah'
            ],
            'repetition_boredom': [
                r'same thing', r'already showed me', r'different', r'something new',
                r'other options', r'what else', r'boring', r'again', r'repeat'
            ],
            'confusion_indicators': [
                r'confused', r'don\'?t understand', r'what do you mean',
                r'huh', r'unclear', r'explain'
            ],
            'distraction_signals': [
                r'whatever', r'doesn\'?t matter', r'any', r'don\'?t care',
                r'sure', r'fine', r'ok'
            ]
        }

    def analyze_message(self, message: str, context: ConversationContext) -> Dict[str, float]:
        """Enhanced message analysis with pattern learning"""
        message_lower = message.lower()
        scores = {}

        logger.debug(f"Analyzing message: '{message_lower}'")

        # Check for positive engagement factors
        for factor, score in self.ENGAGEMENT_FACTORS.items():
            if factor in self.patterns:
                for pattern in self.patterns[factor]:
                    if re.search(pattern, message_lower):
                        scores[factor] = score
                        context.update_engagement_pattern(factor)
                        logger.debug(f"Found {factor}: +{score} points")
                        break

        # Check for negative factors
        for factor, score in self.NEGATIVE_FACTORS.items():
            if factor in self.patterns:
                for pattern in self.patterns[factor]:
                    if re.search(pattern, message_lower):
                        scores[factor] = score
                        logger.debug(f"Found {factor}: {score} points")
                        break

        # Conversation depth bonus
        if context.conversation_depth > 5:
            scores['deep_conversation'] = 10
            logger.debug("Deep conversation bonus: +10 points")

        # Temporal factors
        if context.last_interest_boost:
            time_since_boost = datetime.now() - context.last_interest_boost
            if time_since_boost < timedelta(minutes=2):
                scores['momentum_bonus'] = 5
                logger.debug("Momentum bonus: +5 points")

        # Add basic engagement if no patterns matched
        if not scores:
            scores['basic_engagement'] = 3
            logger.debug("Basic engagement: +3 points")

        return scores

    def calculate_interest_score(self, scores: Dict[str, float], current_score: float) -> float:
        """Calculate new interest score with decay and momentum"""
        score_change = sum(scores.values())

        # Apply score decay over time (simulates natural interest decline)
        decay_factor = 0.98 if current_score > 0 else 1.0
        decayed_score = current_score * decay_factor

        # Calculate new score with bounds
        new_score = max(0, min(100, decayed_score + score_change))

        logger.debug(f"Score: {current_score} -> {new_score} (change: {score_change:+.1f})")

        return new_score

    def extract_preferences_advanced(self, message: str, context: ConversationContext) -> Dict[str, Any]:
        """Advanced preference extraction with context awareness"""
        preferences = {}
        message_lower = message.lower()

        # Food categories with expanded detection
        categories = {
            'burgers': ['burger', 'cheeseburger', 'hamburger'],
            'pizza': ['pizza', 'pie', 'slice'],
            'chicken': ['chicken', 'wings', 'poultry', 'fried chicken'],
            'beef': ['beef', 'steak', 'meat'],
            'fish': ['fish', 'seafood', 'salmon', 'tuna'],
            'salads': ['salad', 'greens', 'healthy'],
            'sandwiches': ['sandwich', 'sub', 'wrap'],
            'desserts': ['dessert', 'sweet', 'cake', 'ice cream'],
            'beverages': ['drink', 'beverage', 'soda', 'juice']
        }

        mentioned_categories = []
        for category, keywords in categories.items():
            if any(keyword in message_lower for keyword in keywords):
                mentioned_categories.append(category)

        if mentioned_categories:
            preferences['preferred_categories'] = mentioned_categories
            context.current_category_focus = mentioned_categories[0]

        # Enhanced flavor and cuisine detection
        flavors = {
            'spicy': ['spicy', 'hot', 'fire', 'jalapeño', 'pepper', 'chili'],
            'mild': ['mild', 'gentle', 'light'],
            'sweet': ['sweet', 'sugary', 'dessert-like'],
            'savory': ['savory', 'salty', 'umami'],
            'fresh': ['fresh', 'crisp', 'light'],
            'rich': ['rich', 'creamy', 'indulgent']
        }

        detected_flavors = []
        for flavor, keywords in flavors.items():
            if any(keyword in message_lower for keyword in keywords):
                detected_flavors.append(flavor)

        if detected_flavors:
            preferences['flavor_preferences'] = detected_flavors

        # Budget extraction with context
        budget_patterns = [
            r'under \$?(\d+)',
            r'less than \$?(\d+)',
            r'budget of \$?(\d+)',
            r'around \$?(\d+)',
            r'about \$?(\d+)',
            r'maximum \$?(\d+)',
            r'up to \$?(\d+)'
        ]

        for pattern in budget_patterns:
            budget_match = re.search(pattern, message_lower)
            if budget_match:
                budget = float(budget_match.group(1))
                preferences['max_budget'] = budget
                context.preferred_price_range = (0.0, budget)
                break

        # Dietary restrictions with expanded detection
        dietary_map = {
            'vegetarian': ['vegetarian', 'veggie', 'no meat'],
            'vegan': ['vegan', 'plant-based'],
            'gluten-free': ['gluten free', 'gluten-free', 'celiac'],
            'dairy-free': ['dairy free', 'dairy-free', 'lactose'],
            'keto': ['keto', 'ketogenic', 'low carb'],
            'healthy': ['healthy', 'light', 'nutritious']
        }

        detected_dietary = []
        for restriction, keywords in dietary_map.items():
            if any(keyword in message_lower for keyword in keywords):
                detected_dietary.append(restriction)

        if detected_dietary:
            preferences['dietary_requirements'] = detected_dietary
            context.dietary_restrictions.extend(detected_dietary)

        return preferences

class DatabaseQueryEngine:
    """
    Advanced database operations with caching and optimization
    """

    def __init__(self, db_path: str = "foodiebot.db"):
        self.db_path = db_path
        self.connection = None
        self.cursor = None
        self.query_cache = {}
        self.cache_timeout = 300  # 5 minutes

    def connect(self):
        """Connect to database with optimizations"""
        if not self.connection:
            self.connection = sqlite3.connect(
                self.db_path,
                check_same_thread=False,
                timeout=10
            )
            self.connection.row_factory = sqlite3.Row
            self.cursor = self.connection.cursor()

            # Enable WAL mode for better concurrency
            self.cursor.execute("PRAGMA journal_mode=WAL")
            self.cursor.execute("PRAGMA synchronous=NORMAL")
            self.cursor.execute("PRAGMA cache_size=10000")

    def disconnect(self):
        """Safely disconnect from database"""
        if self.connection:
            self.connection.close()
            self.connection = None
            self.cursor = None

    def _get_cache_key(self, **kwargs) -> str:
        """Generate cache key from query parameters"""
        return str(sorted(kwargs.items()))

    def search_products_intelligent(self,
                                   category: str = None,
                                   categories: List[str] = None,
                                   max_price: float = None,
                                   min_price: float = None,
                                   dietary_tags: List[str] = None,
                                   mood_tags: List[str] = None,
                                   spice_level_max: int = None,
                                   exclude_ids: List[str] = None,
                                   limit: int = 5,
                                   strategy: str = 'popularity') -> List[Dict[str, Any]]:
        """
        Intelligent product search with multiple strategies
        """
        if not self.connection:
            self.connect()

        # Check cache first
        cache_key = self._get_cache_key(
            category=category, categories=categories, max_price=max_price,
            min_price=min_price, dietary_tags=dietary_tags, mood_tags=mood_tags,
            spice_level_max=spice_level_max, exclude_ids=exclude_ids,
            limit=limit, strategy=strategy
        )

        if cache_key in self.query_cache:
            cache_entry = self.query_cache[cache_key]
            if time.time() - cache_entry['timestamp'] < self.cache_timeout:
                logger.debug("Cache hit for product search")
                return cache_entry['data']

        start_time = time.time()

        # Build dynamic query
        query = "SELECT * FROM products WHERE active = TRUE"
        params = []

        # Category filtering
        if categories:
            placeholders = ",".join(["?"] * len(categories))
            query += f" AND category IN ({placeholders})"
            params.extend(categories)
        elif category:
            query += " AND category = ?"
            params.append(category)

        # Price filtering
        if max_price:
            query += " AND price <= ?"
            params.append(max_price)

        if min_price:
            query += " AND price >= ?"
            params.append(min_price)

        if spice_level_max is not None:
            query += " AND spice_level <= ?"
            params.append(spice_level_max)

        # Exclusion filtering
        if exclude_ids:
            placeholders = ",".join(["?"] * len(exclude_ids))
            query += f" AND product_id NOT IN ({placeholders})"
            params.extend(exclude_ids)

        # JSON field filtering
        if dietary_tags:
            for tag in dietary_tags:
                query += " AND dietary_tags LIKE ?"
                params.append(f'%"{tag}"%')

        if mood_tags:
            for tag in mood_tags:
                query += " AND mood_tags LIKE ?"
                params.append(f'%"{tag}"%')

        # Order strategy
        if strategy == 'popularity':
            query += " ORDER BY popularity_score DESC, price ASC"
        elif strategy == 'price_low':
            query += " ORDER BY price ASC, popularity_score DESC"
        elif strategy == 'price_high':
            query += " ORDER BY price DESC, popularity_score DESC"
        elif strategy == 'random':
            query += " ORDER BY RANDOM()"
        elif strategy == 'newest':
            query += " ORDER BY created_at DESC"

        query += " LIMIT ?"
        params.append(limit)

        try:
            self.cursor.execute(query, params)
            results = []

            for row in self.cursor.fetchall():
                product = dict(row)
                # Parse JSON fields
                for json_field in ['ingredients', 'dietary_tags', 'mood_tags', 'allergens']:
                    try:
                        product[json_field] = json.loads(product[json_field]) if product[json_field] else []
                    except (json.JSONDecodeError, TypeError):
                        product[json_field] = []
                results.append(product)

            # Cache results
            self.query_cache[cache_key] = {
                'data': results,
                'timestamp': time.time()
            }

            query_time = (time.time() - start_time) * 1000
            logger.debug(f"Database query executed in {query_time:.2f}ms")

            return results
        except Exception as e:
            logger.error(f"Database query error: {e}")
            return []

    def get_personalized_recommendations(self,
                                       context: 'ConversationContext',
                                       limit: int = 8) -> List[Dict[str, Any]]:
        """
        Get personalized recommendations using multiple strategies
        """
        recommendations = []

        # Strategy 1: Category-focused recommendations
        if context.current_category_focus:
            category_recs = self.search_products_intelligent(
                category=context.current_category_focus,
                max_price=context.user_preferences.get('max_budget'),
                exclude_ids=context.last_recommendations,
                limit=3
            )
            recommendations.extend(category_recs)

        # Strategy 2: Preference-based recommendations
        if context.user_preferences.get('preferred_categories'):
            pref_recs = self.search_products_intelligent(
                categories=context.user_preferences['preferred_categories'],
                dietary_tags=context.dietary_restrictions,
                exclude_ids=context.last_recommendations + [r['product_id'] for r in recommendations],
                limit=3
            )
            recommendations.extend(pref_recs)

        # Strategy 3: Popular items within budget
        remaining = limit - len(recommendations)
        if remaining > 0:
            all_excluded = context.last_recommendations + [r['product_id'] for r in recommendations]
            popular_recs = self.search_products_intelligent(
                max_price=context.preferred_price_range[1] if context.preferred_price_range[1] < 50 else None,
                exclude_ids=all_excluded,
                strategy='popularity',
                limit=remaining
            )
            recommendations.extend(popular_recs)

        # Strategy 4: Surprise recommendations (random)
        if len(recommendations) < limit:
            remaining = limit - len(recommendations)
            all_excluded = context.last_recommendations + [r['product_id'] for r in recommendations]
            surprise_recs = self.search_products_intelligent(
                exclude_ids=all_excluded,
                strategy='random',
                limit=remaining
            )
            recommendations.extend(surprise_recs)

        # Update recommendation tracking
        new_rec_ids = [r['product_id'] for r in recommendations]
        context.last_recommendations = (context.last_recommendations + new_rec_ids)[-20:]  # Keep last 20

        return recommendations[:limit]

    def log_interaction(self, session_id: str, event_type: str, product_id: str = None, metadata: Dict = None):
        """Log user interactions for analytics"""
        if not self.connection:
            self.connect()

        try:
            self.cursor.execute('''
                INSERT INTO analytics (product_id, event_type, session_id, metadata)
                VALUES (?, ?, ?, ?)
            ''', (product_id, event_type, session_id, json.dumps(metadata or {})))
            self.connection.commit()
        except Exception as e:
            logger.error(f"Error logging interaction: {e}")

class ConversationalAI:
    """
    Advanced conversational AI with adaptive response generation
    """

    def __init__(self, api_key: str, use_groq: bool = True):
        self.use_groq = use_groq
        self.api_key = api_key
        self.client = None
        self.model_performance = {}
        self._initialize_client()

    def _initialize_client(self):
        """Initialize AI client with fallback options"""
        try:
            if self.use_groq:
                self.client = Groq(api_key=self.api_key)
                self.model = "moonshotai/kimi-k2-instruct-0905"  # Using a more standard model
            else:
                import google.generativeai as genai
                genai.configure(api_key=self.api_key)
                self.client = genai.GenerativeModel('gemini-pro')
        except Exception as e:
            logger.error(f"Error initializing AI client: {e}")
            self.client = None

    def generate_response(self,
                         user_message: str,
                         context: ConversationContext,
                         recommended_products: List[Dict[str, Any]] = None,
                         interest_score: float = 0) -> str:
        """Generate contextually aware responses with adaptive tone"""

        if not self.client:
            return "I'm having trouble connecting to my AI service. Please try again later."

        try:
            system_prompt = self._build_adaptive_system_prompt(
                context, recommended_products, interest_score
            )

            messages = []

            if self.use_groq:
                messages.append({"role": "system", "content": system_prompt})

                # Include relevant conversation history
                history_limit = min(6, len(context.conversation_history))
                for msg in context.conversation_history[-history_limit:]:
                    messages.append({"role": "user", "content": msg["user"]})
                    messages.append({"role": "assistant", "content": msg["bot"]})

                messages.append({"role": "user", "content": user_message})

                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=messages,
                    max_tokens=450,
                    temperature=self._get_adaptive_temperature(interest_score),
                    top_p=0.9
                )

                return response.choices[0].message.content

            else:
                prompt = f"{system_prompt}\n\nUser: {user_message}\nFoodieBot:"
                response = self.client.generate_content(prompt)
                return response.text

        except Exception as e:
            logger.error(f"Error generating response: {e}")
            return self._get_fallback_response(interest_score)

    def _get_adaptive_temperature(self, interest_score: float) -> float:
        """Adjust creativity based on interest score"""
        if interest_score > 70:
            return 0.6  # More focused when highly interested
        elif interest_score > 40:
            return 0.8  # Balanced creativity
        else:
            return 0.9  # More creative to re-engage

    def _build_adaptive_system_prompt(self,
                                     context: ConversationContext,
                                     products: List[Dict[str, Any]] = None,
                                     interest_score: float = 0) -> str:
        """Build adaptive system prompt based on conversation state"""

        # Determine conversation phase
        if context.conversation_depth < 3:
            phase = "discovery"
        elif interest_score > 70:
            phase = "closing"
        elif interest_score > 40:
            phase = "exploration"
        else:
            phase = "re-engagement"

        base_prompt = f"""You are FoodieBot, an intelligent food recommendation AI for a fast food restaurant.

CONVERSATION STATE:
- Interest Score: {interest_score:.1f}%
- Phase: {phase}
- Conversation Depth: {context.conversation_depth}
- User Focus: {context.current_category_focus or 'General'}
- Dominant Preferences: {context.get_dominant_preferences()}

BEHAVIORAL GUIDELINES:
1. Be conversational, enthusiastic, and helpful
2. Adapt your approach based on the conversation phase:
   - Discovery: Ask questions to learn preferences
   - Exploration: Present varied options with explanations
   - Closing: Focus on specific recommendations and next steps
   - Re-engagement: Try new angles and exciting options

3. Product Integration:
   - Naturally weave recommendations into conversation
   - Explain WHY products match user needs
   - Highlight unique features and value

4. Tone Adaptation:
   - High interest (70%+): Confident, specific, action-oriented
   - Medium interest (40-70%): Exploratory, informative
   - Low interest (<40%): Engaging, curious, offer variety

"""

        if products:
            base_prompt += f"\nRECOMMENDED PRODUCTS (integrate naturally):\n"
            for i, product in enumerate(products[:3], 1):
                base_prompt += f"{i}. {product['name']} - ${product['price']}\n"
                base_prompt += f"   {product['category']} | Spice: {product['spice_level']}/10 | {product['calories']} cal\n"
                base_prompt += f"   Why: {product['description'][:60]}...\n"

        base_prompt += f"""
RESPONSE REQUIREMENTS:
- Keep responses under 4 sentences for better engagement
- Always explain the 'why' behind recommendations
- Ask follow-up questions to deepen understanding
- Show genuine excitement about food
- Be helpful and never pushy

Current user message requires a {phase} phase response.
"""

        return base_prompt

    def _get_fallback_response(self, interest_score: float) -> str:
        """Generate fallback responses when AI service fails"""
        fallbacks = [
            "I'm having a brief technical moment, but I'm still excited to help you find great food! What sounds good to you?",
            "My brain is catching up, but I'm here to help! What kind of flavors are you in the mood for?",
            "Let me refocus - what's your perfect meal looking like today?",
        ]
        return fallbacks[int(interest_score) % len(fallbacks)]


class FoodieBotPhase3:
    """
    Phase 3: Smart Recommendation & Analytics System
    Extends recommendation logic with collaborative & mood-based filtering,
    budget optimization, dietary intelligence, and rich analytics tracking.
    """

    def __init__(self, db_path: str = "foodiebot.db"):
        self.db_path = db_path

    def recommend_products_advanced(self, context: 'ConversationContext', limit: int = 10) -> List[Dict[str, Any]]:
        """
        Advanced product recommendation combining:
        - Preference matching (category and tags)
        - Collaborative filtering simulation (past popular combos)
        - Mood-based filtering
        - Budget and dietary constraints
        """
        with sqlite3.connect(self.db_path) as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()

            categories = context.user_preferences.get('preferred_categories', [])
            dietary_tags = context.dietary_restrictions
            mood_tags = context.mood_tags or context.user_preferences.get('flavor_preferences', [])
            max_price = context.preferred_price_range[1] if hasattr(context, 'preferred_price_range') else None

            # Base dynamic query with parameters
            query_parts = ["SELECT * FROM products WHERE active = TRUE"]
            params = []

            # Category filter (support multiple)
            if categories:
                query_parts.append(f"category IN ({','.join(['?']*len(categories))})")
                params.extend(categories)

            # Dietary filtering (JSON LIKE match)
            for tag in dietary_tags:
                query_parts.append("dietary_tags LIKE ?")
                params.append(f'%"{tag}"%')

            # Mood filtering (JSON LIKE match)
            for tag in mood_tags:
                query_parts.append("mood_tags LIKE ?")
                params.append(f'%"{tag}"%')

            # Price filtering
            if max_price is not None:
                query_parts.append("price <= ?")
                params.append(max_price)

            # Exclude recently recommended products
            exclude_ids = getattr(context, 'last_recommendations', [])
            if exclude_ids:
                query_parts.append(f"product_id NOT IN ({','.join(['?']*len(exclude_ids))})")
                params.extend(exclude_ids)

            # Compose final query
            query = " AND ".join(query_parts)
            query += " ORDER BY popularity_score DESC, price ASC LIMIT ?"
            params.append(limit)

            cursor.execute(query, params)
            results = []
            for row in cursor.fetchall():
                product = dict(row)
                for json_field in ['ingredients', 'dietary_tags', 'mood_tags', 'allergens']:
                    try:
                        product[json_field] = json.loads(product[json_field]) if product[json_field] else []
                    except (json.JSONDecodeError, TypeError):
                        product[json_field] = []
                results.append(product)

            # Update last recommendations for context
            context.last_recommendations = (exclude_ids + [p['product_id'] for p in results])[-50:]

            return results

    def log_analytics_event(self, session_id: str, product_id: str, event_type: str, metadata: Dict = None):
        """
        Log detailed analytics event to track recommendations, views, clicks, orders.
        """
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO analytics (product_id, event_type, session_id, metadata)
                VALUES (?, ?, ?, ?)
            ''', (
                product_id,
                event_type,
                session_id,
                json.dumps(metadata or {})
            ))
            conn.commit()

    def generate_analytics_report(self, days: int = 30) -> Dict[str, Any]:
        """
        Generate rich analytics including:
        - Interest score progression
        - Top recommended products
        - Highest converting products (orders)
        - Category performance
        - Average conversation & conversion metrics
        """
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()

            # Aggregate recommendations count by product
            cursor.execute('''
                SELECT product_id, COUNT(*) as rec_count
                FROM analytics
                WHERE event_type = 'recommend'
                AND timestamp >= datetime('now', ? || ' days')
                GROUP BY product_id
                ORDER BY rec_count DESC
                LIMIT 10
            ''', (-days,))
            most_recommended = cursor.fetchall()

            # Aggregate conversions count by product
            cursor.execute('''
                SELECT product_id, COUNT(*) as conv_count
                FROM analytics
                WHERE event_type = 'order'
                AND timestamp >= datetime('now', ? || ' days')
                GROUP BY product_id
                ORDER BY conv_count DESC
                LIMIT 10
            ''', (-days,))
            highest_conversion = cursor.fetchall()

            # Category-wise actions
            cursor.execute('''
                SELECT p.category, COUNT(a.id) as action_count
                FROM analytics a
                JOIN products p ON a.product_id = p.product_id
                WHERE a.timestamp >= datetime('now', ? || ' days')
                GROUP BY p.category
                ORDER BY action_count DESC
            ''', (-days,))
            category_performance = cursor.fetchall()

            # Summarize conversation analytics from sessions table
            cursor.execute('''
                SELECT
                    COUNT(*) as total_sessions,
                    AVG(total_messages) as avg_messages,
                    AVG(max_interest_score) as avg_max_interest,
                    AVG(final_interest_score) as avg_final_interest,
                    SUM(CASE WHEN conversion_achieved THEN 1 ELSE 0 END) as conversions
                FROM sessions
                WHERE start_time >= datetime('now', ? || ' days')
            ''', (-days,))
            session_stats = cursor.fetchone()

            conversion_rate = 0
            if session_stats and session_stats[0] > 0:
                conversion_rate = (session_stats[4] / session_stats[0]) * 100

            report = {
                'most_recommended': most_recommended,
                'highest_conversion': highest_conversion,
                'category_performance': category_performance,
                'total_sessions': session_stats[0] if session_stats else 0,
                'avg_messages_per_session': session_stats[1] if session_stats else 0,
                'avg_max_interest_score': session_stats[2] if session_stats else 0,
                'avg_final_interest_score': session_stats[3] if session_stats else 0,
                'conversion_rate_percent': round(conversion_rate, 2),
                'conversions': session_stats[4] if session_stats else 0
            }

            return report

    def get_session_analytics(self, session_id: str):
        """
        Retrieve analytics data specific to a user session, e.g. interest score progression,
        product views, recommendations, and conversions.
        """
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.cursor()
            cursor.execute('''
                SELECT timestamp, event_type, metadata
                FROM analytics
                WHERE session_id = ?
                ORDER BY timestamp ASC
            ''', (session_id,))
            return cursor.fetchall()


class FoodieBotConversationManager:
    """
    Master conversation manager with advanced orchestration
    """

    def __init__(self, groq_api_key: str, db_path: str = "foodiebot.db"):
        self.db_path = db_path
        self.groq_api_key = groq_api_key

        # Initialize core components
        self.interest_engine = InterestScoringEngine()
        self.db_engine = DatabaseQueryEngine(db_path)
        self.ai_engine = ConversationalAI(groq_api_key, use_groq=True)

        # Session management
        self.active_sessions: Dict[str, ConversationContext] = {}
        self.session_cleanup_interval = 3600  # 1 hour
        self.last_cleanup = time.time()

        # Performance metrics
        self.metrics = {
            'total_conversations': 0,
            'successful_recommendations': 0,
            'average_interest_score': 0.0,
            'session_duration_avg': 0.0,
            'conversion_rate': 0.0
        }

        # Initialize database connection
        self._initialize_database()
        logger.info("FoodieBotConversationManager initialized successfully")

    def _initialize_database(self):
        """Initialize database with required tables"""
        try:
            self.db_engine.connect()

            # Create tables if they don't exist
            self.db_engine.cursor.execute('''
                CREATE TABLE IF NOT EXISTS products (
                    product_id TEXT PRIMARY KEY,
                    name TEXT NOT NULL,
                    category TEXT NOT NULL,
                    price REAL NOT NULL,
                    description TEXT,
                    ingredients TEXT,
                    dietary_tags TEXT,
                    mood_tags TEXT,
                    allergens TEXT,
                    spice_level INTEGER DEFAULT 0,
                    calories INTEGER,
                    popularity_score REAL DEFAULT 0.0,
                    active BOOLEAN DEFAULT TRUE,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')

            self.db_engine.cursor.execute('''
                CREATE TABLE IF NOT EXISTS analytics (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    product_id TEXT,
                    event_type TEXT NOT NULL,
                    session_id TEXT NOT NULL,
                    metadata TEXT,
                    timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    FOREIGN KEY (product_id) REFERENCES products (product_id)
                )
            ''')

            self.db_engine.cursor.execute('''
                CREATE TABLE IF NOT EXISTS sessions (
                    session_id TEXT PRIMARY KEY,
                    start_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    end_time TIMESTAMP,
                    total_messages INTEGER DEFAULT 0,
                    max_interest_score REAL DEFAULT 0.0,
                    final_interest_score REAL DEFAULT 0.0,
                    recommended_products TEXT,
                    user_preferences TEXT,
                    conversion_achieved BOOLEAN DEFAULT FALSE
                )
            ''')

            self.db_engine.connection.commit()
            logger.info("Database initialized successfully")

        except Exception as e:
            logger.error(f"Database initialization failed: {e}")

    def get_or_create_session(self, session_id: str) -> ConversationContext:
        """Get existing session or create new one"""
        if session_id not in self.active_sessions:
            self.active_sessions[session_id] = ConversationContext()
            self.metrics['total_conversations'] += 1

            # Log new session in database
            try:
                self.db_engine.cursor.execute('''
                    INSERT INTO sessions (session_id, start_time)
                    VALUES (?, ?)
                ''', (session_id, datetime.now()))
                self.db_engine.connection.commit()
            except Exception as e:
                logger.error(f"Error logging new session: {e}")

        return self.active_sessions[session_id]

    def process_message(self,
                       user_message: str,
                       session_id: str,
                       user_metadata: Dict[str, Any] = None) -> Dict[str, Any]:
        """
        Main message processing pipeline with comprehensive orchestration
        """
        try:
            # Clean up old sessions periodically
            self._cleanup_old_sessions()

            # Get or create conversation context
            context = self.get_or_create_session(session_id)
            context.conversation_depth += 1
            context.last_response_time = datetime.now()

            # Log interaction
            self.db_engine.log_interaction(
                session_id,
                'user_message',
                metadata={'message': user_message, 'metadata': user_metadata}
            )

            # Step 1: Analyze user message for engagement and preferences
            engagement_scores = self.interest_engine.analyze_message(user_message, context)
            new_preferences = self.interest_engine.extract_preferences_advanced(user_message, context)

            # Step 2: Update interest score
            old_interest = context.current_interest_score
            context.current_interest_score = self.interest_engine.calculate_interest_score(
                engagement_scores, context.current_interest_score
            )

            # Track interest boosts
            if context.current_interest_score > old_interest + 10:
                context.last_interest_boost = datetime.now()

            # Step 3: Update user preferences and context
            if new_preferences:
                context.user_preferences.update(new_preferences)
                logger.debug(f"Updated preferences: {new_preferences}")

            # Step 4: Get personalized product recommendations
            recommended_products = []
            if context.current_interest_score > 20:  # Only recommend when there's some interest
                recommended_products = self.db_engine.get_personalized_recommendations(
                    context, limit=5
                )

                if recommended_products:
                    # Log recommendations
                    self.db_engine.log_interaction(
                        session_id,
                        'recommendations_shown',
                        metadata={'products': [p['product_id'] for p in recommended_products]}
                    )

            # Step 5: Generate AI response
            bot_response = self.ai_engine.generate_response(
                user_message,
                context,
                recommended_products,
                context.current_interest_score
            )

            # Step 6: Update conversation history
            context.conversation_history.append({
                'user': user_message,
                'bot': bot_response,
                'timestamp': datetime.now().isoformat(),
                'interest_score': context.current_interest_score,
                'engagement_factors': list(engagement_scores.keys())
            })

            # Keep conversation history manageable
            if len(context.conversation_history) > 50:
                context.conversation_history = context.conversation_history[-40:]

            # Step 7: Update session metrics
            self._update_session_metrics(session_id, context)

            # Step 8: Prepare response data
            response_data = {
                'bot_response': bot_response,
                'interest_score': context.current_interest_score,
                'conversation_depth': context.conversation_depth,
                'recommended_products': recommended_products,
                'user_preferences': context.user_preferences,
                'engagement_factors': engagement_scores,
                'session_metrics': self._get_session_summary(context),
                'conversation_phase': self._determine_conversation_phase(context)
            }

            # Log successful response
            self.db_engine.log_interaction(
                session_id,
                'bot_response',
                metadata={
                    'interest_score': context.current_interest_score,
                    'response_length': len(bot_response),
                    'products_recommended': len(recommended_products)
                }
            )

            return response_data

        except Exception as e:
            logger.error(f"Error processing message: {e}")
            return {
                'bot_response': "I apologize, but I'm experiencing some technical difficulties. Let me try to help you anyway! What kind of food are you interested in?",
                'interest_score': 0,
                'error': str(e)
            }

    def _determine_conversation_phase(self, context: ConversationContext) -> str:
        """Determine current conversation phase for analytics"""
        if context.conversation_depth < 3:
            return "discovery"
        elif context.current_interest_score > 70:
            return "closing"
        elif context.current_interest_score > 40:
            return "exploration"
        else:
            return "re-engagement"

    def _update_session_metrics(self, session_id: str, context: ConversationContext):
        """Update session metrics in database"""
        try:
            session_duration = (datetime.now() - context.session_start_time).total_seconds()

            self.db_engine.cursor.execute('''
                UPDATE sessions SET
                    total_messages = ?,
                    max_interest_score = CASE
                        WHEN ? > max_interest_score THEN ?
                        ELSE max_interest_score
                    END,
                    final_interest_score = ?,
                    recommended_products = ?,
                    user_preferences = ?
                WHERE session_id = ?
            ''', (
                context.conversation_depth,
                context.current_interest_score, context.current_interest_score,
                context.current_interest_score,
                json.dumps([p for p in context.last_recommendations[-10:]]),  # Last 10 recommendations
                json.dumps(context.user_preferences),
                session_id
            ))
            self.db_engine.connection.commit()

            # Update global metrics
            self.metrics['average_interest_score'] = (
                (self.metrics['average_interest_score'] * (self.metrics['total_conversations'] - 1) +
                 context.current_interest_score) / self.metrics['total_conversations']
            )

        except Exception as e:
            logger.error(f"Error updating session metrics: {e}")

    def _get_session_summary(self, context: ConversationContext) -> Dict[str, Any]:
        """Get summary statistics for current session"""
        session_duration = (datetime.now() - context.session_start_time).total_seconds()

        return {
            'duration_seconds': session_duration,
            'messages_exchanged': context.conversation_depth,
            'interest_score': context.current_interest_score,
            'recommendations_shown': len(context.last_recommendations),
            'preferences_learned': len(context.user_preferences),
            'dominant_patterns': sorted(
                context.user_engagement_patterns.items(),
                key=lambda x: x[1],
                reverse=True
            )[:3]
        }

    def _cleanup_old_sessions(self):
        """Clean up inactive sessions to free memory"""
        if time.time() - self.last_cleanup < self.session_cleanup_interval:
            return

        current_time = datetime.now()
        sessions_to_remove = []

        for session_id, context in self.active_sessions.items():
            if context.last_response_time:
                inactive_duration = current_time - context.last_response_time
                if inactive_duration.total_seconds() > 3600:  # 1 hour inactive
                    sessions_to_remove.append(session_id)

        for session_id in sessions_to_remove:
            # Archive session data before removing
            self._archive_session(session_id)
            del self.active_sessions[session_id]

        self.last_cleanup = time.time()
        if sessions_to_remove:
            logger.info(f"Cleaned up {len(sessions_to_remove)} inactive sessions")

    def _archive_session(self, session_id: str):
        """Archive session data when cleaning up"""
        try:
            context = self.active_sessions[session_id]
            session_duration = (datetime.now() - context.session_start_time).total_seconds()

            self.db_engine.cursor.execute('''
                UPDATE sessions SET
                    end_time = ?,
                    conversion_achieved = ?
                WHERE session_id = ?
            ''', (
                datetime.now(),
                context.current_interest_score > 70,  # Consider high interest as conversion
                session_id
            ))
            self.db_engine.connection.commit()

        except Exception as e:
            logger.error(f"Error archiving session {session_id}: {e}")

    def get_analytics_summary(self, days: int = 7) -> Dict[str, Any]:
        """Get analytics summary for specified period"""
        try:
            cutoff_date = datetime.now() - timedelta(days=days)

            # Get session statistics
            self.db_engine.cursor.execute('''
                SELECT
                    COUNT(*) as total_sessions,
                    AVG(total_messages) as avg_messages,
                    AVG(max_interest_score) as avg_max_interest,
                    AVG(final_interest_score) as avg_final_interest,
                    SUM(CASE WHEN conversion_achieved THEN 1 ELSE 0 END) as conversions
                FROM sessions
                WHERE start_time > ?
            ''', (cutoff_date,))

            session_stats = dict(self.db_engine.cursor.fetchone())

            # Get popular products
            self.db_engine.cursor.execute('''
                SELECT
                    p.name,
                    p.category,
                    COUNT(*) as recommendation_count
                FROM analytics a
                JOIN products p ON a.product_id = p.product_id
                WHERE a.event_type = 'recommendations_shown'
                AND a.timestamp > ?
                GROUP BY a.product_id
                ORDER BY recommendation_count DESC
                LIMIT 10
            ''', (cutoff_date,))

            popular_products = [dict(row) for row in self.db_engine.cursor.fetchall()]

            # Calculate conversion rate
            conversion_rate = 0
            if session_stats['total_sessions'] > 0:
                conversion_rate = (session_stats['conversions'] / session_stats['total_sessions']) * 100

            return {
                'period_days': days,
                'total_sessions': session_stats['total_sessions'] or 0,
                'avg_messages_per_session': round(session_stats['avg_messages'] or 0, 1),
                'avg_max_interest_score': round(session_stats['avg_max_interest'] or 0, 1),
                'avg_final_interest_score': round(session_stats['avg_final_interest'] or 0, 1),
                'conversion_rate': round(conversion_rate, 1),
                'conversions': session_stats['conversions'] or 0,
                'popular_products': popular_products,
                'active_sessions': len(self.active_sessions)
            }

        except Exception as e:
            logger.error(f"Error generating analytics: {e}")
            return {'error': str(e)}

    def end_session(self, session_id: str) -> Dict[str, Any]:
        """Gracefully end a conversation session"""
        if session_id not in self.active_sessions:
            return {'error': 'Session not found'}

        context = self.active_sessions[session_id]

        # Archive session
        self._archive_session(session_id)

        # Generate session summary
        summary = self._get_session_summary(context)

        # Remove from active sessions
        del self.active_sessions[session_id]

        return {
            'session_summary': summary,
            'message': 'Session ended successfully'
        }

    def get_session_status(self, session_id: str) -> Dict[str, Any]:
        """Get current status of a session"""
        if session_id not in self.active_sessions:
            return {'error': 'Session not found'}

        context = self.active_sessions[session_id]
        return self._get_session_summary(context)

# Streamlit Interface (if available)
def create_streamlit_interface():
    """Create Streamlit interface for FoodieBot"""
    if not STREAMLIT_AVAILABLE:
        logger.warning("Streamlit not available - skipping interface creation")
        return

    st.set_page_config(
        page_title="FoodieBot",
        page_icon="🍔",
        layout="wide"
    )

    st.title("🍔 FoodieBot - Intelligent Food Recommendations")

    # Initialize session state
    if 'conversation_manager' not in st.session_state:
        groq_api_key = st.secrets.get("GROQ_API_KEY", os.getenv("GROQ_API_KEY"))
        if not groq_api_key:
            st.error("Please set GROQ_API_KEY in secrets or environment variables")
            st.stop()

        st.session_state.conversation_manager = FoodieBotConversationManager(groq_api_key)
        st.session_state.conversation_manager.add_sample_products()
        st.session_state.session_id = f"session_{int(time.time())}"
        st.session_state.messages = []

    # Sidebar with session info
    with st.sidebar:
        st.header("Session Info")

        if st.session_state.session_id in st.session_state.conversation_manager.active_sessions:
            session_status = st.session_state.conversation_manager.get_session_status(
                st.session_state.session_id
            )
            st.metric("Interest Score", f"{session_status.get('interest_score', 0):.1f}%")
            st.metric("Messages", session_status.get('messages_exchanged', 0))
            st.metric("Duration", f"{session_status.get('duration_seconds', 0):.0f}s")

        if st.button("New Session"):
            st.session_state.session_id = f"session_{int(time.time())}"
            st.session_state.messages = []
            st.rerun()

        if st.button("View Analytics"):
            analytics = st.session_state.conversation_manager.get_analytics_summary()
            st.json(analytics)

    # Chat interface
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.write(message["content"])

            if message["role"] == "assistant" and "products" in message:
                if message["products"]:
                    st.subheader("🍽️ Recommended for You:")
                    for product in message["products"][:3]:
                        with st.expander(f"{product['name']} - ${product['price']}"):
                            st.write(f"**Category:** {product['category']}")
                            st.write(f"**Description:** {product['description']}")
                            st.write(f"**Calories:** {product['calories']}")
                            st.write(f"**Spice Level:** {product['spice_level']}/10")

    # Chat input
    if prompt := st.chat_input("What kind of food are you craving?"):
        # Display user message
        with st.chat_message("user"):
            st.write(prompt)

        st.session_state.messages.append({"role": "user", "content": prompt})

        # Get bot response
        with st.chat_message("assistant"):
            with st.spinner("Thinking about delicious options..."):
                response_data = st.session_state.conversation_manager.process_message(
                    prompt,
                    st.session_state.session_id
                )

            st.write(response_data['bot_response'])

        # Store assistant message
        st.session_state.messages.append({
            "role": "assistant",
            "content": response_data['bot_response'],
            "products": response_data.get('recommended_products', [])
        })

# Main execution
if __name__ == "__main__":
    if STREAMLIT_AVAILABLE and len(os.sys.argv) > 1 and os.sys.argv[1] == "streamlit":
        create_streamlit_interface()
    else:
        # Command line interface
        groq_api_key = "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"
        if not groq_api_key:
            print("Please set GROQ_API_KEY environment variable")
            exit(1)

        manager = FoodieBotConversationManager(groq_api_key)
        #manager.add_sample_products()

        session_id = f"cli_session_{int(time.time())}"
        print("🍔 FoodieBot CLI - Type 'quit' to exit, 'analytics' for stats")
        print("=" * 50)

        while True:
            try:
                user_input = input("\nYou: ").strip()

                if user_input.lower() in ['quit', 'exit', 'q']:
                    summary = manager.end_session(session_id)
                    print(f"\n📊 Session Summary: {summary['session_summary']}")
                    break

                if user_input.lower() == 'analytics':
                    analytics = manager.get_analytics_summary()
                    print(f"\n📈 Analytics: {json.dumps(analytics, indent=2)}")
                    continue

                if not user_input:
                    continue

                response_data = manager.process_message(user_input, session_id)
                print(f"\n🤖 FoodieBot: {response_data['bot_response']}")
                print(f"📊 Interest Score: {response_data['interest_score']:.1f}%")

            except KeyboardInterrupt:
                print("\n\nGoodbye! 👋")
                break
            except Exception as e:
                print(f"Error: {e}")
                continue


🍔 FoodieBot CLI - Type 'quit' to exit, 'analytics' for stats

You: hey, i need something for family reunion


/tmp/ipython-input-328741386.py:1022: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  self.db_engine.cursor.execute('''



🤖 FoodieBot: Hey there! How fun - family reunions are the BEST time to break out crowd-pleasing food! Are you thinking finger foods people can grab while mingling, or more substantial stuff for a sit-down meal? And roughly how many people are we talking about?
📊 Interest Score: 20.0%

You: something more of a hearty meal you know,comforting vibes?

🤖 FoodieBot: Oh perfect, comfort food it is! For that cozy family reunion vibe, I'm thinking our Crispy Chicken Fusion Burger - it's got that perfect nostalgic crunch but with a gourmet twist that'll have everyone asking for seconds! The juicy chicken with melty cheese on a toasted brioche just screams "home cooking" but elevated. How many relatives are we feeding, and are there any picky eaters or dietary needs I should know about?
📊 Interest Score: 48.6%

You: nah,we are asian family

🤖 FoodieBot: Ah amazing! For an Asian family gathering, let me suggest our Bold Everything Seasoning Breakfast - it's got that perfect umami-rich flavor pro

/tmp/ipython-input-328741386.py:1259: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  self.db_engine.cursor.execute('''
/tmp/ipython-input-328741386.py:1273: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  self.db_engine.cursor.execute('''



📈 Analytics: {
  "period_days": 7,
  "total_sessions": 2,
  "avg_messages_per_session": 7.5,
  "avg_max_interest_score": 98.9,
  "avg_final_interest_score": 98.9,
  "conversion_rate": 50.0,
  "conversions": 1,
  "popular_products": [],
  "active_sessions": 1
}

You: recommended products

🤖 FoodieBot: Top picks for your sweet-and-spicy lineup:  
1. Crispy Maple Syrup Wings – $6.42, 2/10 spice, 7/10 sweet, 4.6★  
2. Ultimate Pulled-Pork Pizza – $11.26, 2/10 spice, sweet BBQ sauce, 4.5★  
3. Zesty Arugula Salad – $14.60, 0 spice, bright citrus dressing, 4.4★  

All crowd-pleasers under the $60 budget—want me to bundle any combo for your 8-person order?
📊 Interest Score: 100.0%

You: thanks

🤖 FoodieBot: You’re welcome! Just let me know when you’re ready to lock in those maple wings or if you’d like any tweaks—happy to hold the order.
📊 Interest Score: 100.0%

You: quit

📊 Session Summary: {'duration_seconds': 212.897117, 'messages_exchanged': 12, 'interest_score': 100, 'recommendations_s

/tmp/ipython-input-328741386.py:1238: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  self.db_engine.cursor.execute('''
